In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE67698'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of CFBE41o-ΔF508 cells and CFBE41o−CFTR cells."
!Series_summary	"Cystic fibrosis bronchial epithelial (CFBE41o-ΔF508) cells subjected to 23 bio-active small molecules including vehicle controls, at low temperature and untreated cells. Untreated Cystic fibrosis bronchial epithelial cells (CFBE41o−CFTR) are also included."
!Series_overall_design	"Two-colors Dye-swap, two or three biological replicates with two technical replicates each and DMSO (vehicle control) were included in each hybridization batch."
Sample Characteristics Dictionary:
{0: ['cell line: polarized CFBE41o-cell line'], 1: ['transduction: TranzVector lentivectors containing deltaF508 CFTR (CFBE41o-deltaF508CFTR)', 'transduction: TranzVector lentivectors containing wildtype CFTR (CFBE41o-CFTR)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine Gene Expression Data Availability
# More rigorous check to determine gene expression data availability based on sample characteristics content
sample_characteristics = {0: ['cell line: polarized CFBE41o-cell line'], 
                          1: ['transduction: TranzVector lentivectors containing deltaF508 CFTR (CFBE41o-deltaF508CFTR)', 
                              'transduction: TranzVector lentivectors containing wildtype CFTR (CFBE41o-CFTR)']}
if any('CFBE41o' in characteristic for characteristics in sample_characteristics.values() for characteristic in characteristics):
    is_gene_available = True

# Data Availability
# Since the study is on cell lines, no human data for Cystic Fibrosis, age, or gender.

def dummy_conversion(value):
    return None

convert_trait = dummy_conversion
convert_age = dummy_conversion
convert_gender = dummy_conversion

save_cohort_info('GSE67698', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None)
